# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

**Примечания:**

Для оценки качества моделей следует применять метрику RMSE.

Необходимо самостоятельно освоить библиотеку LightGBM и её средствами построить модели градиентного бустинга.

Время выполнения ячейки кода Jupyter Notebook можно получить специальной командой. Необходимо найти её.

Поскольку модель градиентного бустинга может обучаться долго, изменить у неё следует только два-три параметра.

Если перестанет работать Jupyter Notebook, можно удалить лишние переменные оператором *del* `del features_train`

**Описание данных**

Данные находятся в файле /datasets/autos.csv.

**Признаки:**
- DateCrawled — дата скачивания анкеты из базы
- VehicleType — тип автомобильного кузова
- RegistrationYear — год регистрации автомобиля
- Gearbox — тип коробки передач
- Power — мощность (л. с.)
- Model — модель автомобиля
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- FuelType — тип топлива
- Brand — марка автомобиля
- NotRepaired — была машина в ремонте или нет
- DateCreated — дата создания анкеты
- NumberOfPictures — количество фотографий автомобиля
- PostalCode — почтовый индекс владельца анкеты (пользователя)
- LastSeen — дата последней активности пользователя

**Целевой признак:**

Price — цена (евро)

**Цель проекта.**

Создать модель, которая будет определять стоимость автомобиля, при этом ключевыми факторами является качество (определяемое метрикой RMSE), а таке скорость обучения и скорость предсказания.

**План работы.**

1. Загрузить данные и провести предобработку.
2. Провести обучение моделей с использованием различных наборов гиперпараметров.
3. Провести анализ скорости и качества работы моделей, выбрать лучшую модель по результатам метрики RMSE и времени работы.
4. Оформить проект и написать выводы.

## Подготовка данных

Предварительно загрузим необходимые для работы библиотеки. 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures, OrdinalEncoder, OneHotEncoder, StandardScaler 
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from catboost import Pool, CatBoostRegressor, cv
from lightgbm import LGBMRegressor
from sklearn.pipeline import Pipeline
warnings.filterwarnings('ignore')
import time

Теперь закрузим датасет и взглянем на него.

In [2]:
try:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/autos.csv')
except:
    df = pd.read_csv('D:/learn_data/autos.csv')

display(df.info())
display(df.describe())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

None

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Как мы видим, в ряде признаков имеются как пропуски, так и явные аномалии, как например мощность 20000 л.с. Необходимо провести предобработку. Проверять датасет на дубликаты в данном случае не имеет смысла, поскольку среди более 300 тысяч объектов, вполне могут найтись автомобили с одинаковыми признаками и продаваемых по одной цене, а идентификационных признаки конкретного автомобиля у нас не имеется. Для начала взглянем на период создания анкет, которые мы используем для обучения моделей.

In [3]:
date = pd.to_datetime(df['DateCreated'])
print(date.min())
print(date.max())

2014-03-10 00:00:00
2016-04-07 00:00:00


Как мы видим, база данных содежит анкеты, созданные в период с 2014 по 2016 год, с учетом того, что мы в 2023 году на основе этих данных планируем обучить модели, предсказывающие цену автомобиля, примем во внимание тот факт, что в данной работе не учитывается инфляция и другие параметры, влиющие на цену автомобиля с учетом времени (в реальной работе, полагаю, данный факт должен будет учитываться, но для нас это прощает дело).

Определим, какие признаки влияют на цену автомобиля (сам прзнак цены - `Price` нам естественно также необходим):
- VehicleType. Тип машины говорит о ее функционале, таким образом влияет на ее цену. 
- RegistrationYear. Год регистрации определяет сколько уже лет машина на ходу, то есть, чем больше ее возраст, тем меньше цена.
- Gearbox. Наличие автоматической коробки передач повышает стоимость автомобиля из-за сложности конструкции и простоты управления.
- Power. Мощность в лошадиных силах (далее л.с.) напрямую влияет на цену авто.
- Model. В купе с типом машины и брендом также может влиять на популярность у покупателей.
- Kilometer. Километраж обратно пропорционально влияет на цену, поскольку, чем больше километров проехала машина, тем больше ее износ и соответсвенно меньше цена.
- FuelType. Тип топлива определяет конструкцию двигателя внутреннего сгорания и также имеет влияние на цену.
- Brand. Компания, которая создала автомобиль, может указывать качество и надежность автомобиля, также имеет зачение бренд.
- Repaired. Также влияет на цену машины, поскольку авто, которые побывали в дорожно-транспортных проишествиях, подвергались ремонту, а следовательно изменениям и их надежность ниже.

Остальные столбцы - DateCrawled (дата скачивания анкеты из базы), RegistrationMonth (месяц регистраци авто), DateCreated (дата создания анкеты), NumberOfPictures (количество фтографий авто), PostalCode (почтовый индекс владельца анкеты), LastSeen (дата посленей активости пользователя) для модели не требуются, так как не влияют на стоимость.

In [4]:
df = df.drop(['DateCrawled', 'RegistrationMonth', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen'], axis=1) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             354369 non-null  int64 
 1   VehicleType       316879 non-null  object
 2   RegistrationYear  354369 non-null  int64 
 3   Gearbox           334536 non-null  object
 4   Power             354369 non-null  int64 
 5   Model             334664 non-null  object
 6   Kilometer         354369 non-null  int64 
 7   FuelType          321474 non-null  object
 8   Brand             354369 non-null  object
 9   Repaired          283215 non-null  object
dtypes: int64(4), object(6)
memory usage: 27.0+ MB


Теперь взглянем на пропуски в столбцах.

In [5]:
df.isna().mean()

Price               0.000000
VehicleType         0.105794
RegistrationYear    0.000000
Gearbox             0.055967
Power               0.000000
Model               0.055606
Kilometer           0.000000
FuelType            0.092827
Brand               0.000000
Repaired            0.200791
dtype: float64

In [6]:
df['Repaired'].value_counts()

no     247161
yes     36054
Name: Repaired, dtype: int64

Теперь проведем замену пропусков в параметрах `Gearbox` (будем логично считать, что если пользователь не указал тип коробки передач как автоматический, значит стоит механическая), `Repaired` (будем считать, что не указание сведений о ремонте авто, указывает на отсутствие такового), `Model` (с учетом того, что модель нам неизвестна, так и обозначим ее).

In [7]:
df.fillna({'Gearbox':'manual',
           'Repaired':'no',
           'Model':'unknown'},
          inplace=True)
df.isna().mean()

Price               0.000000
VehicleType         0.105794
RegistrationYear    0.000000
Gearbox             0.000000
Power               0.000000
Model               0.000000
Kilometer           0.000000
FuelType            0.092827
Brand               0.000000
Repaired            0.000000
dtype: float64

Что касается пропусков в типе кузова и типе топлива, с учетом отсутствия данных, предлагаем оставшиеся пропуски удалить.

In [8]:
df['VehicleType'].unique()

array([nan, 'coupe', 'suv', 'small', 'sedan', 'convertible', 'bus',
       'wagon', 'other'], dtype=object)

In [9]:
df['FuelType'].unique()

array(['petrol', 'gasoline', nan, 'lpg', 'other', 'hybrid', 'cng',
       'electric'], dtype=object)

In [10]:
df.fillna({'VehicleType':'other',
           'FuelType':'other'},
          inplace=True)
df.isna().mean()

Price               0.0
VehicleType         0.0
RegistrationYear    0.0
Gearbox             0.0
Power               0.0
Model               0.0
Kilometer           0.0
FuelType            0.0
Brand               0.0
Repaired            0.0
dtype: float64

Теперь взглянем на данные целевого признака цены.

In [11]:
df['Price'].value_counts()

0        10772
500       5670
1500      5394
1000      4649
1200      4594
         ...  
1368         1
233          1
11080        1
16340        1
10985        1
Name: Price, Length: 3731, dtype: int64

Избавимся от аномальных показателей, таких как цена в 0 евро, а также слишком дорогих авто. Примем за данность, что будем обучать модели на показателях автомобилей дороже 350 евро и дешевле 10000 евро. Также избавимся от аномальных показателей в мощности двигателя, возьмем для обучения моделей сведения об автомобилях с мощностью от 25 до 400 л.с.

In [12]:
df = df.query('250 <= Price <= 20000')
df = df.query('24 < Power < 401')

df['Price'].describe()

count    300346.000000
mean       4890.967238
std        4575.484500
min         250.000000
25%        1400.000000
50%        3200.000000
75%        6999.000000
max       20000.000000
Name: Price, dtype: float64

Придадим значениям показателей `Gearbox` и `Repaired` буллев тип (будем считать авто с ручной коробкой передач как **1**, а с автоматической как **0**, также будем считать как **1** то, что машина была в ремонте и **0**, если не была. Для этих значений создадим новые столбцы `IsManualGearbox` и `IsRepaired` с соответствующими параметрами, а стерые столбцы удалим.

In [13]:
df['IsManualGearbox'] = 0
df.loc[df['Gearbox'] == 'manual', 'IsManualGearbox'] = 1
display(df['IsManualGearbox'].value_counts())
df['Gearbox'].value_counts()

1    240809
0     59537
Name: IsManualGearbox, dtype: int64

manual    240809
auto       59537
Name: Gearbox, dtype: int64

In [14]:
df['IsRepaired'] = 0
df.loc[df['Repaired'] == 'yes', 'IsRepaired'] = 1
display(df['IsRepaired'].value_counts())
df['Repaired'].value_counts()

0    272920
1     27426
Name: IsRepaired, dtype: int64

no     272920
yes     27426
Name: Repaired, dtype: int64

In [15]:
df.drop(['Repaired','Gearbox'], inplace=True, axis=1)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300346 entries, 1 to 354368
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             300346 non-null  int64 
 1   VehicleType       300346 non-null  object
 2   RegistrationYear  300346 non-null  int64 
 3   Power             300346 non-null  int64 
 4   Model             300346 non-null  object
 5   Kilometer         300346 non-null  int64 
 6   FuelType          300346 non-null  object
 7   Brand             300346 non-null  object
 8   IsManualGearbox   300346 non-null  int64 
 9   IsRepaired        300346 non-null  int64 
dtypes: int64(6), object(4)
memory usage: 25.2+ MB


**Вывод:** В ходе работы мы изучили таблицу данных, заполнили по возможности пропуски или удалили их, также избавились от аномальных показателей.

## Обучение моделей

Выберем следующие регрессоры для обучения модели:

1. LinearRegression
2. Ridge
3. DecisionTreeRegressor
4. CatBoostRegressor
5. LGBMRegressor

В качестве метрики для всех моделей будем использовать MSE и в финальной таблице переведем в RMSE. Качество моделей будем проверять на кроссвалидации.

Подготовим три типа признаков для кроссвалидации (будут применяться нами в зависимости от модели):

Неизмененные. Название `features`.
Закодированные методом OHE. Название `features_ohe`.
Закодированные методом OE. Название `features_ord`.
Целевые признаки везде одинаковые. Название `target`.

Сначала разобьем сет на признаки и целевой признак, а затем выборки на обучающую, валидационную и тестовую.

In [17]:
features = df.drop('Price', axis=1)
target = df['Price']

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.4, random_state=3301)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_test, target_test, test_size=0.5, random_state=3301)

In [18]:
print(features_train.shape, target_train.shape)
print(features_valid.shape, target_valid.shape)
print(features_test.shape, target_test.shape)

(180207, 9) (180207,)
(60069, 9) (60069,)
(60070, 9) (60070,)


Мы разбили на выборки в соотншении 3:1:1

Теперь также создадим датасеты, в которых категориальные данные будут приведены к числовым. Определим признаки, которые надо закодировать. 

In [19]:
cat_columns = ['VehicleType', 'FuelType','Brand','Model']

Теперь применим прямое кодирование. Обучение проведем на тренировочной выборке, с помощью которого затем проведем трансформацию и других выборок.

In [20]:
features_ohe_train = features_train.copy()
encoder_ohe = OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False)
encoder_ohe.fit(features_train[cat_columns])

features_ohe_train[encoder_ohe.get_feature_names_out()] = encoder_ohe.transform(features_ohe_train[cat_columns])
features_ohe_train = features_ohe_train.drop(cat_columns, axis=1)

features_ohe_valid = features_valid.copy()
features_ohe_valid[encoder_ohe.get_feature_names_out()] = encoder_ohe.transform(features_ohe_valid[cat_columns])
features_ohe_valid = features_ohe_valid.drop(cat_columns, axis=1)

features_ohe_test = features_test.copy()
features_ohe_test[encoder_ohe.get_feature_names_out()] = encoder_ohe.transform(features_ohe_test[cat_columns])
features_ohe_test = features_ohe_test.drop(cat_columns, axis=1)

Теперь применим порядковое кодирование. Обучение проведем на тренировочной выборке, с помощью которого затем проведем трансформацию и других выборок.

In [21]:
encoder_ord = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1 )
encoder_ord.fit(features_train)

features_ord_train = pd.DataFrame(encoder_ord.transform(features_train), columns=features_train.columns)
features_ord_valid = pd.DataFrame(encoder_ord.transform(features_valid), columns=features_valid.columns)
features_ord_test = pd.DataFrame(encoder_ord.transform(features_test), columns=features_test.columns)

Теперь приступим собственно к обучению моделей и определению RMSE.

### Линейная регрессия

In [22]:
%%time

regressor = LinearRegression()
print('# Обучение модели и определение RMSE')
print()
cv_RMSE_LR = (cross_val_score(regressor, 
                             features_ohe_train, 
                             target_train, 
                             cv=5, 
                             scoring='neg_mean_squared_error').mean() * -1) ** 0.5
print('Среднее RMSE на кроссвалидации LinearRegression =', cv_RMSE_LR)

# Обучение модели и определение RMSE

Среднее RMSE на кроссвалидации LinearRegression = 2806.6684968493323
Wall time: 20.1 s


Среднее значение RMSE на кроссвалидаци LinearRegression составило более *2806.6684968493323*, что довольно плохо (не должно превышать 2500). 

### Регрессия Rigle

In [23]:
%%time

regressor = Ridge()
hyperparams = [{'solver':['auto', 'svd', 'cholesky', 'lsqr','sparse_cg']}]


print('# Настройка параметров, обучение модели и определение RMSE')
print()
clf = GridSearchCV(regressor, hyperparams, scoring='neg_mean_squared_error')
clf.fit(features_ohe_train, target_train)
print("Лучший показатель RMSE при параметрах:")
print()
print(clf.best_params_)
print()
print("Сетка оценок RMSE:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.6f для %r" % ((mean * -1) ** 0.5, params))
print()

cv_RMSE_R = (max(means) * -1) ** 0.5

# Настройка параметров, обучение модели и определение RMSE

Лучший показатель RMSE при параметрах:

{'solver': 'svd'}

Сетка оценок RMSE:

2806.947640 для {'solver': 'auto'}
2806.947640 для {'solver': 'svd'}
2806.947640 для {'solver': 'cholesky'}
4205.555915 для {'solver': 'lsqr'}
3363.023886 для {'solver': 'sparse_cg'}

Wall time: 1min 6s


Лучшее значение RMSE регресси Ridge составило *2806.947640* при параметре `svd`. 

### Регрессия DecisionTreeRegressor

In [44]:
%%time

regressor = DecisionTreeRegressor() 
max_depth_list = [x for x in range(2, 31)]
hyperparams = [{'criterion':['mse'], 
                'max_depth':max_depth_list, 
                'random_state':[3301]}]

print('# Настройка гиперпараметров, обучение модели и определение RMSE (здесь придется немного подождать, у меня заняло чуть более 9 минут)')
print()
clf = GridSearchCV(regressor, hyperparams, scoring='neg_mean_squared_error')
clf.fit(features_ohe_train, target_train)
print("Лучший показатель RMSE при параметрах:")
print()
print(clf.best_params_)
print()
print("Сетка оценок RMSE:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.6f для %r" % ((mean * -1) ** 0.5, params))
print()

cv_RMSE_DTR = (max(means) * -1) ** 0.5

# Настройка гиперпараметров, обучение модели и определение RMSE (здесь придется немного подождать, у меня заняло чуть более 9 минут)

Лучший показатель RMSE при параметрах:

{'criterion': 'mse', 'max_depth': 17, 'random_state': 3301}

Сетка оценок RMSE:

3268.803815 для {'criterion': 'mse', 'max_depth': 2, 'random_state': 3301}
2979.595717 для {'criterion': 'mse', 'max_depth': 3, 'random_state': 3301}
2680.086181 для {'criterion': 'mse', 'max_depth': 4, 'random_state': 3301}
2465.729615 для {'criterion': 'mse', 'max_depth': 5, 'random_state': 3301}
2311.380128 для {'criterion': 'mse', 'max_depth': 6, 'random_state': 3301}
2192.206349 для {'criterion': 'mse', 'max_depth': 7, 'random_state': 3301}
2111.006493 для {'criterion': 'mse', 'max_depth': 8, 'random_state': 3301}
2051.476763 для {'criterion': 'mse', 'max_depth': 9, 'random_state': 3301}
1999.012365 для {'criterion': 'mse', 'max_depth': 10, 'random_state': 3301}
1957.510820 для {'criterion': 'mse', 'max_depth': 11, 'random_state':

Лучшее значение RMSE регресси DecisionTreeRegressor составило *1883.774832* при максимальной глубине дерева `17`. Что уже лучше требуемой.

### Регрессия CatBoostRegressor

In [25]:
%%time

regressor = CatBoostRegressor(learning_rate=0.5,
                              iterations = 200,
                              random_state=3301, 
                              verbose=False, 
                              cat_features=['VehicleType', 'FuelType','Brand','Model']) 
print('# Обучение модели и определение RMSE')
print()
cv_RMSE_CBR = (cross_val_score(regressor,
                                features_train, 
                                target_train, 
                                cv=5, 
                                scoring='neg_mean_squared_error').mean() * -1) ** 0.5
print('Среднее RMSE на кроссвалидации CatBoostRegressor =', cv_RMSE_CBR)

# Обучение модели и определение RMSE

Среднее RMSE на кроссвалидации CatBoostRegressor = 1638.0046738212843
Wall time: 1min 37s


Среднее значение RMSE на кроссвалидаци CatBoostRegression составило *1638.0046738212843*, что заметно лучше предыдущих моделей.

### Регрессия LGBMRegressor

In [26]:
%%time

regressor = LGBMRegressor() 
hyperparams = [{'num_leaves':[31, 100, 200], 
                'learning_rate':[0.1, 0.3, 0.5],
                'random_state':[3301]}]

print('# Настройка гиперпараметров, обучение модели и определение RMSE')
print()
clf = GridSearchCV(regressor, hyperparams, scoring='neg_mean_squared_error')
clf.fit(features_ord_train, target_train)
print("Лучший показатель RMSE при параметрах:")
print()
print(clf.best_params_)
print()
print("Сетка оценок RMSE:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.6f для %r" % ((mean * -1) ** 0.5, params))
print()

cv_RMSE_LGBMR = (max(means) * -1) ** 0.5

# Настройка гиперпараметров, обучение модели и определение RMSE

Лучший показатель RMSE при параметрах:

{'learning_rate': 0.3, 'num_leaves': 200, 'random_state': 3301}

Сетка оценок RMSE:

1711.201729 для {'learning_rate': 0.1, 'num_leaves': 31, 'random_state': 3301}
1618.186045 для {'learning_rate': 0.1, 'num_leaves': 100, 'random_state': 3301}
1579.829698 для {'learning_rate': 0.1, 'num_leaves': 200, 'random_state': 3301}
1642.000657 для {'learning_rate': 0.3, 'num_leaves': 31, 'random_state': 3301}
1589.215292 для {'learning_rate': 0.3, 'num_leaves': 100, 'random_state': 3301}
1577.204345 для {'learning_rate': 0.3, 'num_leaves': 200, 'random_state': 3301}
1646.117694 для {'learning_rate': 0.5, 'num_leaves': 31, 'random_state': 3301}
1612.549308 для {'learning_rate': 0.5, 'num_leaves': 100, 'random_state': 3301}
1610.118497 для {'learning_rate': 0.5, 'num_leaves': 200, 'random_state': 3301}

Wall time: 30.6 s


Лучшее значение RMSE регресси LGBMRegressor составило *1577.204345* при показателях learning_rate' равном  0.3 и 'num_leaves' равном 200. На данный момент это лучший показатель RMSE.

**Вывод:** Нами были изучены 5 регрессионных моделей, при этом мы зафиксировали лучшие гиперпараметры и время обучения моделей.

## Анализ моделей

Теперь приступим к обучению моделей на лучших параметрах и проверке показателей RMSE на тестовой выборке, зафиксируем время обучения и проверки моделей, а также показатели RMSE на тестовой выборке, а затем проведем сравнение моделей.

### LinearRegression

In [27]:
%%time

model = LinearRegression()
a = time.time()
model.fit(features_ohe_train, target_train)
b = time.time()
LR_fit = b - a
LR_fit

Wall time: 3.85 s


3.8536791801452637

In [28]:
%%time
a = time.time()
target_predict = model.predict(features_ohe_valid)
b = time.time()
LR_predict = b - a
LR_predict

Wall time: 484 ms


0.48446202278137207

In [29]:
final_RMSE_LR = mean_squared_error(target_valid, target_predict) ** 0.5
final_RMSE_LR

2821.324772613799

### Ridge

In [30]:
%%time

model = Ridge(solver='svd')
a = time.time()
model.fit(features_ohe_train, target_train)
b = time.time()
R_fit = b - a
R_fit

Wall time: 7.2 s


7.201472997665405

In [31]:
%%time
a = time.time()

target_predict = model.predict(features_ohe_valid)
b = time.time()
R_predict = b - a
R_predict

Wall time: 124 ms


0.1240077018737793

In [32]:
final_RMSE_R = mean_squared_error(target_valid, target_predict) ** 0.5
final_RMSE_R

2821.621751452996

### DecisionTreeRegressor

In [33]:
%%time

model = DecisionTreeRegressor(criterion='mse', 
                              max_depth=17, 
                              random_state=3301) 
a = time.time()

model.fit(features_ohe_train, target_train)
b = time.time()
DTR_fit = b - a
DTR_fit

Wall time: 5.84 s


5.836105585098267

In [34]:
%%time
a = time.time()

target_predict = model.predict(features_ohe_valid)
b = time.time()
DTR_predict = b - a
DTR_predict

Wall time: 130 ms


0.1290130615234375

In [35]:
final_RMSE_DTR = mean_squared_error(target_valid, target_predict) ** 0.5                                    
final_RMSE_DTR

1841.2529278017075

### CatBoostRegressor

In [36]:
%%time

model = CatBoostRegressor(learning_rate=0.5, 
                          iterations = 200, 
                          random_state=3301, 
                          verbose=False,
                          cat_features=['VehicleType','FuelType','Brand','Model']) 
a = time.time()

model.fit(features_train, target_train)
b = time.time()
CBR_fit = b - a
CBR_fit

Wall time: 22.2 s


22.01894474029541

In [37]:
%%time
a = time.time()

target_predict = model.predict(features_valid)
b = time.time()
CBR_predict = b - a
CBR_predict

Wall time: 159 ms


0.15888547897338867

In [38]:
final_RMSE_CBR = mean_squared_error(target_valid, target_predict) ** 0.5
final_RMSE_CBR

1646.2041313772083

### LGBMRegressor

In [39]:
%%time

model = LGBMRegressor(learning_rate=0.3, 
                      num_leaves=200, 
                      random_state=3301)
a = time.time()

model.fit(features_ord_train, target_train)
b = time.time()
LGBMR_fit = b - a
LGBMR_fit

Wall time: 1.59 s


1.5681226253509521

In [40]:
%%time
a = time.time()

target_predict = model.predict(features_ord_valid)
b = time.time()
LGBMR_predict = b - a
LGBMR_predict

Wall time: 128 ms


0.1280066967010498

In [41]:
final_RMSE_LGBMR = mean_squared_error(target_valid, target_predict) ** 0.5
final_RMSE_LGBMR

1582.1922115632049

### Сравнение моделей

Теперь сведем все полученные данные в таблицу и сравним их (я понимаю, что скорости выполнения кода будут несколько различаться, но не придумал, как можно сохранить это время в переменных и использовать их в таблице, но в целом, думаю, пропорции должны примерно оставаться такими же).

In [42]:
index = ['LinearRegression',
         'Ridge',
         'DecisionTreeRegressor',
         'CatBoostRegressor',
         'LGBMRegressor']
data = {'RMSE на CV':[cv_RMSE_LR,
                      cv_RMSE_R,
                      cv_RMSE_DTR,
                      cv_RMSE_CBR,
                      cv_RMSE_LGBMR],
        'RMSE модели на валидацинной выборке':[final_RMSE_LR,
                                           final_RMSE_R,
                                           final_RMSE_DTR,
                                           final_RMSE_CBR,
                                           final_RMSE_LGBMR],
        'Время обучения модели, сек':[LR_fit,
                                     R_fit,
                                     DTR_fit,
                                     CBR_fit,
                                     LGBMR_fit],
        'Время предсказания модели, сек':[LR_predict,
                                          R_predict, 
                                          DTR_predict,
                                          CBR_predict,
                                          LGBMR_predict]}

scores_data = pd.DataFrame(data=data, index=index)
scores_data['Рейтинг RMSE'] = (scores_data['RMSE модели на валидацинной выборке'].min() /
                              scores_data['RMSE модели на валидацинной выборке'])
scores_data['Рейтинг времени обучения'] = (scores_data['Время обучения модели, сек'].min() / 
                              scores_data['Время обучения модели, сек'])
scores_data['Рейтинг времени предсказания'] = (scores_data['Время предсказания модели, сек'].min() / 
                              scores_data['Время предсказания модели, сек'])
scores_data['Итоговый рейтинг'] = (scores_data['Рейтинг RMSE'] +
                                   scores_data['Рейтинг времени обучения'] +
                                   scores_data['Рейтинг времени предсказания'])
scores_data

,RMSE на CV,RMSE модели на валидацинной выборке,"Время обучения модели, сек","Время предсказания модели, сек",Рейтинг RMSE,Рейтинг времени обучения,Рейтинг времени предсказания,Итоговый рейтинг
LinearRegression,2806.668497,2821.324773,3.853679,0.484462,0.560798,0.406916,0.255970,1.223683
Ridge,2806.947640,2821.621751,7.201473,0.124008,0.560739,0.217750,1.000000,1.778489
DecisionTreeRegressor,1883.774832,1841.252928,5.836106,0.129013,0.859302,0.268693,0.961203,2.089198
CatBoostRegressor,1638.004674,1646.204131,22.018945,0.158885,0.961115,0.071217,0.780485,1.812817
LGBMRegressor,1577.204345,1582.192212,1.568123,0.128007,1.000000,1.000000,0.968759,2.968759


### Контрольная проверка лидера

In [43]:
%%time
a = time.time()

target_predict = model.predict(features_ord_test)
b = time.time()
LGBMR_control = b - a
LGBMR_control
control_RMSE_LGBMR = mean_squared_error(target_test, target_predict) ** 0.5
control_RMSE_LGBMR

Wall time: 127 ms


1546.1710503319566

Итак, на тестовой выборке лучшая модель показала результат RMSE *1546.1710503319566*, что лучше требуемой в поставленной задаче, и довольно неплохую скорость.

**Вывод:**
Итак, как очевидно следует из таблицы безусловным лидером стала модель LGBMRegressor, как показавшая наилучшую скорость обучения, практически лучшую скорость предсказания и также лучший результат по метрике RMSE. Модель CatBoostRegressor также показала неплохие результаты по качеству и скорости предсказания, но существенно уступает лидеру в скорости обучения. DecisionTreeRegressor показал относительно небольшую скорость обучения и предсказания, однако качество несколько уступает как лидеру, так и модели CatBoostRegressor, что, на мой взгляд, говорит о его не слишком большой успешности. Остальные две модели не прошли даже по требованию о показатели RMSE ниже *2500*.

## **Общие выводы:**

В ходе работы были осуществлены загрузка, изучение и предобработка данных, сравнение моделей с использованием различных наборов гиперпараметров, выбрана лучшая модель по результатам метрики RMSE, времени обучения и предсказания. Для общей оценки всем трем параметрам (поскольку специально это не оговорено в задании, полагаем параметры качества, скорости обучения и предсказания равнозначными) был создан относительный рейтинг. Первое место получает модель с самым минмальным временем и RMSE. Остальные получают рейтинг как отношение лучшего в столбце к своему значению. На выбор модели не влияло время обучения на кроссвалидации, так как на одном типе данных ее проводят один раз. В то время переобучить модель на новых данных приходится намного чаще.

Лучшей моделью по всем трем параметрам *качество*, *скорость обучения* и *скорость предсказания* безусловно стала модель LGBMRegressor, показатель RMSE на тестовой выборке которой составил *1546.1710503319566*, а также наименьшую скорость обучения и предсказания. Сочетания скорости и точности на подготовленных данных выводит его в лидеры рейтинга. Все остальные модели показали себя в целом хуже.